In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
from pathlib import Path

In [74]:
def create_package(nlp):
    meta=nlp.meta
    pkgs = Path("../pkgs")
    tmpd = Path("/tmp") / meta["name"]
    nlp.to_disk(tmpd)
    
    package(tmpd, pkgdir, force=True)
    model_name  =  meta["lang"] + "_" + meta["name"]
    pkgd = pkgs / (model_name+ "-" + meta["version"])
    return pkgd, tmpd

# date_ruler

In [75]:
import spacy
from bedoner.lang.mecab import Japanese
name="date_ruler"
nlp = Japanese(meta={"name": "date_ruler"})
p = nlp.create_pipe(name)
nlp.add_pipe(p)
pkgd, tmpd = create_package(nlp)

nlp = spacy.load(tmpd)
nlp("2019年11月8日に高松隆と海に行った").ents

✔ Loaded meta.json from file
/tmp/date_ruler/meta.json
✔ Successfully created package 'mecab_date_ruler-0.0.0'
../pkgs/mecab_date_ruler-0.0.0
To build the package, run `python setup.py sdist` in this directory.


(2019年11月8日,)

# person_ruler

- mecabのユーザ辞書を含める必要がありちょっと面倒

In [76]:
import os 
import spacy
from bedoner.lang.mecab import Japanese
from bedoner.entity_rulers.person import create_person_ruler
from pathlib import Path
from spacy.cli import package
from shutil import copy

name="person_ruler"
user_dic = os.path.expanduser("~/.bedoner/user.dic")
nlp = Japanese(meta={"tokenizer": {"userdic": user_dic}, "name":"person_ruler"})
nlp.add_pipe(create_person_ruler(nlp))
pkgd, tmpd = create_package(nlp)

✔ Loaded meta.json from file
/tmp/person_ruler/meta.json
✔ Successfully created package 'mecab_person_ruler-0.0.0'
../pkgs/mecab_person_ruler-0.0.0
To build the package, run `python setup.py sdist` in this directory.


もろもろのデータの追加，setup.pyとMANIFEST.inの修正

In [88]:
dst = (pkgd/"jinmei").absolute()
if dst.exists():
    os.unlink(str(dst))
os.symlink(Path("./jinmei").absolute(), (pkgd/"jinmei").absolute())
copy("setup.mecab_template.py", str(pkgd/"setup.py"))
with (pkgd/"MANIFEST.in").open("a") as f:
    f.write("recursive-include jinmei *")

あとはpkgsにいって， `python setup.py sdist`とやればよい

# entity_ruler

In [15]:
import spacy
from bedoner.entity_rulers.person import create_person_ruler

name="entity_ruler"
nlp = Japanese(meta={"name": name})
nlp.add_pipe(DateRuler(nlp))
nlp.add_pipe(create_person_ruler(nlp))
nlp.to_disk(pkgdir / name)

NameError: name 'DateRuler' is not defined

In [20]:
nlp = spacy.load(pkgdir / name) 
nlp("2019年11月8日に高松隆と海に行った").ents

(2019年11月8日, 高松隆)

# knp entity extractor

In [11]:
import spacy
from bedoner.lang.knp import Japanese
from bedoner.entity_extractors.knp import KnpEntityExtractor
name = "knp_entity_extractor"
nlp = Japanese(meta={"name": name})
p = nlp.create_pipe("knp_entity_extractor")
nlp.add_pipe(p)
nlp.to_disk(pkgdir/name)

In [12]:
nlp = spacy.load(pkgdir / name) 
for e in nlp("2019年11月8日に高松隆と海に行った").ents:
    print(e.label_)

DATE
PERSON
